In [1]:
from torchvision import transforms
import glob
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import models
import torch.nn as nn
from torch import optim

# Image transformations
image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ]),
    # Validation does not use augmentation
    'valid':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    # Test does not use augmentation
    'test':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [6]:
batch_size = 100

# Datasets from folders
data = {
    'train':
    datasets.ImageFolder(root="../recipe-box/data_for_nn/train/", transform=image_transforms['train']),
    'valid':
    datasets.ImageFolder(root="../recipe-box/data_for_nn/valid/", transform=image_transforms['valid']),
    'test':
    datasets.ImageFolder(root="../recipe-box/data_for_nn/test/", transform=image_transforms['valid']),
}

# Dataloader iterators, make sure to shuffle
dataloaders = {
    'train': DataLoader(data['train'], batch_size=batch_size, shuffle=True),
    'valid': DataLoader(data['valid'], batch_size=batch_size, shuffle=True),
    'test': DataLoader(data['test'], batch_size=batch_size, shuffle=True)
}

print( 'length of train set:', len(data['train']) )
print( 'length of valid set:', len(data['valid']) )
print( 'length of test set:', len(data['test']) )
print( 'a train sample:', data['train'][0] )

RuntimeError: Found 0 files in subfolders of: ../recipe-box/data_for_nn/train/
Supported extensions are: .jpg,.jpeg,.png,.ppm,.bmp,.pgm,.tif,.tiff,.webp

In [3]:
# Iterate through the dataloader once
trainiter = iter(dataloaders['train'])
features, labels = next(trainiter)
features.shape, labels.shape

(torch.Size([100, 3, 224, 224]), torch.Size([100]))

In [4]:
def get_pretrained_model(model_name, train_on_gpu, n_classes = 100):
    """Retrieve a pre-trained model from torchvision

    Params
    -------
        model_name (str): name of the model (currently only accepts vgg16 and resnet50)

    Return
    --------
        model (PyTorch model): cnn

    """
    model = None
    
    if model_name == 'vgg16':
        model = models.vgg16(pretrained=True)

        # Freeze early layers
        for param in model.parameters():
            param.requires_grad = False
        n_inputs = model.classifier[6].in_features
        # Add on classifier
        model.classifier[6] = nn.Sequential(
            nn.Linear(n_inputs, 256), 
            nn.ReLU(), 
            nn.Dropout(0.2),
            nn.Linear(256, n_classes), 
            nn.LogSoftmax(dim=1))

    elif model_name == 'resnet50':
        model = models.resnet50(pretrained=True)

        for param in model.parameters():
            param.requires_grad = False

        n_inputs = model.fc.in_features
        model.fc = nn.Sequential(
            nn.Linear(n_inputs, 256), 
            nn.ReLU(), 
            nn.Dropout(0.2),
            nn.Linear(256, n_classes), 
            nn.LogSoftmax(dim=1))

    # Move to gpu and parallelize
    if train_on_gpu:
        model = model.to('cuda')
        model = nn.DataParallel(model)

    return model

In [5]:
model = get_pretrained_model('vgg16', train_on_gpu = False)
if multi_gpu:
    summary(
        model.module,
        input_size=(3, 224, 224),
        batch_size=batch_size,
        device='cuda')
else:
    summary(
        model, input_size=(3, 224, 224), batch_size=batch_size, device='cuda')


NameError: name 'multi_gpu' is not defined

In [6]:
# Loss and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters())

n_epochs = 5
for epoch in range(n_epochs):
    for data, targets in dataloaders["train"]:
        # Generate predictions
        out = model(data)
        # Calculate loss
        loss = criterion(out, targets)
        # Backpropagation
        loss.backward()
        # Update model parameters
        optimizer.step()

KeyboardInterrupt: 

In [19]:
# IMG_DIM = (224,224)

# train_files = glob.glob('recipe-box/data_for_nn/train/*')
# train_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in train_files]
# train_imgs = np.array(train_imgs)
# train_labels = [fn.split('\\')[1].split('.')[0].strip() for fn in train_files]

# validation_files = glob.glob('recipe-box/data_for_nn/valid/*')
# validation_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in validation_files]
# validation_imgs = np.array(validation_imgs)
# validation_labels = [fn.split('\\')[1].split('.')[0].strip() for fn in validation_files]

# print('Train dataset shape:', train_imgs.shape, 
#       '\tValidation dataset shape:', validation_imgs.shape)

Train dataset shape: (0,) 	Validation dataset shape: (0,)


In [13]:
# train_imgs_scaled = train_imgs.astype('float32')
# validation_imgs_scaled  = validation_imgs.astype('float32')
# train_imgs_scaled /= 255
# validation_imgs_scaled /= 255

# print(train_imgs[0].shape)
# array_to_img(train_imgs[0])

IndexError: index 0 is out of bounds for axis 0 with size 0

In [22]:
from torchvision import models
model = models.vgg16(pretrained=True)

In [5]:
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [14]:
# Freeze model weights
for param in model.parameters():
    param.requires_grad = False

NameError: name 'model' is not defined

In [24]:
import torch.nn as nn
n_inputs = 4096
n_classes = 100
# Add on classifier
model.classifier[6] = nn.Sequential(
                      nn.Linear(n_inputs, 256), 
                      nn.ReLU(), 
                      nn.Dropout(0.4),
                      nn.Linear(256, n_classes),                   
                      nn.LogSoftmax(dim=1))

In [25]:
# Only training classifier[6]
model.classifier
# nn.Sequential(
#     nn.Linear(in_features=25088, out_features=4096, bias=True),
#     nn.ReLU(inplace=True),
#     nn.Dropout(p=0.5),
#     nn.Linear(in_features=4096, out_features=4096, bias=True),
#     nn.ReLU(inplace=True),
#     nn.Dropout(p=0.5)
#     Sequential(
#         nn.Linear(in_features=4096, out_features=256, bias=True),
#         nn.ReLU(),
#         nn.Dropout(p=0.4),
#         nn.Linear(in_features=256, out_features=100, bias=True),
#         nn.LogSoftmax()
#   )
# )

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.4, inplace=False)
    (3): Linear(in_features=256, out_features=100, bias=True)
    (4): LogSoftmax()
  )
)

In [26]:
# Find total parameters and trainable parameters
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

134,352,036 total parameters.
91,492 training parameters.


In [33]:
dataloaders["train"]

In [32]:
import numpy as np
# Early stopping details
n_epochs_stop = 5
min_val_loss = np.Inf
epochs_no_improve = 0

# Main loop
for epoch in range(n_epochs_stop):
    # Initialize validation loss for epoch
    val_loss = 0
    dataloader_iterator = iter(dataloaders["train"])

    for i, data1 in enumerate(dataloaders2)):

        try:
            data2 = next(dataloader_iterator)
        except StopIteration:
            dataloader_iterator = iter(dataloaders1)
            data2 = next(dataloader_iterator)
  # Training loop
    for data, targets in dataloaders["train"]:
        # Generate predictions
        out = model(data)
        # Calculate loss
        loss = criterion(out, targets)
        # Backpropagation
        loss.backward()
        # Update model parameters
        optimizer.step()
    
# Validation loop
for data, targets in validloader:
    # Generate predictions 
    out = model(data)
    # Calculate loss
    loss = criterion(out, targets)
    val_loss += loss

# Average validation loss
val_loss = val_loss / len(trainloader)

# If the validation loss is at a minimum
if val_loss < min_val_loss:
    # Save the model
    torch.save(model, checkpoint_path)
    epochs_no_improve = 0
    min_val_loss = val_loss

else:
    epochs_no_improve += 1
    # Check early stopping condition
    if epochs_no_improve == n_epochs_stop:
        print('Early stopping!')

    # Load in the best model
    model = torch.load(checkpoint_path)

RuntimeError: [enforce fail at CPUAllocator.cpp:64] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 1284505600 bytes. Error code 12 (Cannot allocate memory)
